_This notebook is part of the material for the [Tutorials for ML4HEP](https://gitlab.com/hepcedar/mcnet-schools/zakopane-2022) session._

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ramonpeter/ML4HEP-Tutorial/blob/main/tensorflow.ipynb)

# Basics of Tensorflow

Tensorflow is a widely used library for machine learning, especially deep learning, both training and inference (evaluating trained neural networks on new data).
It was developed by the Google Brain team, and is open source software.
On the [website](https://www.tensorflow.org) you will find many libraries and tools for common tasks related to machine learning, and a lot of training material and examples.

In this first part, we will get familiar with basic concepts of [Tensorflow](https://www.tensorflow.org).

## Fundamental classes and concepts

The  `Tensor` class can be used for many things be just like a [numpy](https://numpy.org) array (see [this guide](https://www.tensorflow.org/guide/tensor) for some more advanced uses).

In [ ]:
import tensorflow as tf
x1 = tf.linspace(0., 1., 11)
print(x1)

To construct a `Tensor` from its value(s), the `tf.constant` helper method can be used:

In [ ]:
x0 = tf.constant(3.14)
x2 = tf.constant([ [ 1., 2.], [3., 4. ] ])
print(x1)
print(x2)

As you can see, the default type is a 32-bit floating point number. We can also construct a 32-bit integer type `Tensor`:

In [ ]:
x3 = tf.constant([ 1, 1, 2, 3, 5, 8, 13, 25 ])
print(x3)

These are all tensors with a static shape, like in numpy, but it is also possible to make tensors with dynamic shape, if the size in one dimension is not known beforehand.
This is used for input nodes, which can be constructed before the batch size is known:

In [ ]:
x4 = tf.keras.layers.Input(shape=(3,))
print(x4)

It is also possible to make tensors where one dimension changes from entry to entry, and to place a `Tensor` on a GPU.

For weights in the neural network, the `Variable` class should be used, which is essentially a `Tensor` with extra functionality: the values can be stored to and loaded from a file, and it is possible to calculate derivatives with respect to a `Variable`.

### Computation graphs

There is an important feature about tensors which is the original reason they were developed: they can be used to construct a *computation graph*, which represents the calculations that need to be done, without actually performing them.
Since Tensorflow 2.0, the default is the "eager" mode, where the calculation is performed immediately, but constructing graphs (and optimizing them) provides better performance, so this is used by the high-level helper functions to construct neural networks.

It requires a bit of setup, but we can use [Tensorboard](https://www.tensorflow.org/tensorboard) to visualize such graphs (the example below is taken from [here](https://www.tensorflow.org/tensorboard/graphs#graphs_of_tffunctions) in the documentation, for an interactive version you can also view it directly there).

In [ ]:
%load_ext tensorboard
# If not running on Colab, you may need to tell the extension
# where to find the tensorboard executable like this.
# To find the path: activate the conda / virtual environment,
# and run `which tensorboard`
# %env TENSORBOARD_BINARY=/home/pieter/miniconda3/envs/ceci_mltf/bin/tensorboard

In [ ]:
# 3x3 random matrics
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

@tf.function
def f1(a, b):
    return tf.nn.relu(tf.matmul(a, b))

tf.summary.trace_on(graph=True, profiler=True)
z = f1(x, y)
from datetime import datetime
logdir = f'logs/func/{datetime.now().strftime("%Y%m%d-%H%M%S")}'
with tf.summary.create_file_writer(logdir).as_default():
    tf.summary.trace_export(
        name="f1_trace_xy",
        step=0,
        profiler_outdir=logdir)

# %tensorboard --logdir logs/func ## uncomment to launch tensorboard

If everything is set up correctly, this will show the interactive version of tensorboad (alternatively, you can view it at the bottom of [this page](https://www.tensorflow.org/tensorboard/graphs#graphs_of_tffunctions)).

Tensorflow graphs can be optimized depending for different environments, e.g. for inference (evaluation of the neural network outupts) on mobile devices, see [this overview](https://www.tensorflow.org/guide/graph_optimization).
An additional speedup can be obtained by using the [XLA](https://www.tensorflow.org/xla) compiler, which will generate optimized computation kernels specifically for the model you use.

The `tf.function` helper function converts a python function to a Tensorflow computation graph, more details about that are given in [this guide](https://www.tensorflow.org/guide/function).

### Automatic differentiation

Having computation graphs at hand, they can be used for the automatic calculation of derivatives, or gradients.
In a nutshell, training a neural network means minimizing a loss function, which needs the derivatives of the loss function with respect to the network parameters.

With the loss function being, generally, of the form $L_{total} = \sum_i L(f(x_i), y_i)$, calculating the derivative with respect to a generic parameter $w$ of the neural network will require, because of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), the derivative of the network output(s) to that parameter ($j$ loops over the output nodes):
$$
\left.\frac{d\,L(f(x), y)}{d\,w}\right|_{f(x), y} = \sum_j \left.\frac{d\,L}{d\,f_j}\right|_{f_j(x), y}\cdot \left.\frac{d\,f_j}{d\,w}\right|_{x}.
$$

Having a sequential model, which is equivalent to a function $f(x) = f^n \circ f^{n-1} \circ \cdots \circ f^1 (x)$ (where $x$ are the input features and $n$ is the number of layers).
The derivate of the output node $j$ to a parameter in layer $r$ is then given by
$$
\left.\frac{d\,f_j}{d\,w}\right|_{x} = \sum_{k_1} \sum_{k_2} \cdots \sum_{k_{n-r}}
\left.\frac{d\,f^n_j}{d\,z^1_{k_1}}\right|_{f^{n-1}(x)}\cdot
\left.\frac{d\,f^{n-1}_{k_1}}{d\,z^2_{k_2}}\right|_{f^{n-2}(x)}
\cdots
\left.\frac{d\,f^{r+1}_{k_{n-r+1}}}{d\,z^{n-r}_{k_{n-r}}}\right|_{f^{r-1}(x)}\cdot
\left.\frac{d\,f^r_{k_{n-r}}}{d\,w}\right|_{f^{r}(x)}.
$$

Calculating the gradient of the neural network with respect to one of its parameters is called [backpropagation](https://en.wikipedia.org/wiki/Backpropagation).
It is a very efficient way to calculate all the derivatives, since each derivative can easily be computed from the weights and the values of each node.

Tensorflow implements [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) (a more detailed description can be found on [this documentation page](https://www.tensorflow.org/guide/autodiff)):

In [ ]:
x = tf.Variable(2.0) #needs to be a float
with tf.GradientTape() as tape:
    y = x**2
dydx = tape.gradient(y, x)
print(dydx.numpy())

As mentioned before, trainable parameters should be a `Variable` instances, because computation graphs can change the values of a `Variable`, but not of a `Tensor` (this allows to make a single graph for a training step, or even for the whole training of a neural network).

## High-level interfaces

So far we have focused on the core concepts, but for many common tasks, there are helper functions and classes in the [keras](https://keras.io) library, which is now included in Tensorflow as `tensorflow.keras` module.
There is a wealth of documentation and examples on its website, and in the Tensorflow documentation [tf.keras](https://www.tensorflow.org/guide/keras/overview) 

The core class is [`Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model), which represents a neural network, and provides a lot of useful common functionality (e.g. using predefined minimizers, splitting out a test dataset, keeping track of training and test loss).

Building a neural network is very easy with the [sequential model](https://www.tensorflow.org/guide/keras/sequential_model): all we need to specify the number of nodes in each layer, the activation function, and the type of layer. A simple regression model looks like this:

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

x0 = tf.linspace(-8., 8., 2048)
x_learning = tf.reshape(tf.random.shuffle(x0), (2048, 1))
y_learning = ( tf.sin(x_learning)
              + tf.random.normal(x_learning.shape)*.1 )

# construct the model
nn1 = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x : .1*x),
    tf.keras.layers.Dense(128, activation="relu",
                          bias_initializer="glorot_uniform"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
    tf.keras.layers.Lambda(lambda x : 2.2*x-1.1)
])

Tricks used:
- `bias_initializer="glorot_uniform"` initializes the biases with random values; normally this is only done for the weights, and here it really helps (it will help the different nodes to look at different pieces of the curve).
- Since the sigmoid will only asymptotically tend to 1, we scale it by 1.1, such that it can reach the extreme values of the sine function,
- and we scale the inputs by a factor ten such that they are of the order 1. 

Training and plotting the results can then be obtained by (output is turned off with `verbose=0`, otherwise we would get 500 lines printed with the progress of the training):

In [ ]:
# construct graph and train
optimizer = tf.keras.optimizers.SGD(learning_rate=0.085)
nn1.compile(optimizer=optimizer, loss="mse")
nn1.fit(x_learning, y_learning, batch_size=32, epochs=250,
          validation_split=.2, verbose=0)

# plot
plt.scatter(x_learning, y_learning, label="Training data", s=1)
plt.plot(x0, tf.sin(x0), label="y(x)", color="k")
plt.plot(x0, nn1(tf.reshape(x0, (2048,1))),
         label="learned y(x)", color="orange")
plt.legend()

Plotting the training curves:

In [ ]:
plt.plot(nn1.history.history["loss"], label="Training loss")
plt.plot(nn1.history.history["val_loss"], label="Validation loss")
plt.legend()

For more complex layouts, there is also the [functional API](https://www.tensorflow.org/guide/keras/functional), which allows to construct a neural network by applying functions on a set of input nodes (the `Model` is then constructed with a list of input and output nodes, and behaves the same otherwise).